In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import os
import logging
import math
from sklearn.model_selection import KFold
from timeit import default_timer as timer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from scipy.special import beta
from math import factorial as fact
from run_hls import get_perf, execute_hls
from pareto import getParetoFrontier, checkParetoOptimal, getProbabilityOfEval
from generate_directives import RandomDirectiveGenerator, DirectiveCrossover, DirectiveMutator, DirectiveWriter
from multiprocessing import Process
import re
import random
import scipy
import pprint
import pickle

In [15]:
def get_row(df, row):
    if len(df) == 0:
        # return empty if input is empty
        return pd.DataFrame()
    else:
        srs = pd.Series(np.full(len(df), True))
        for col in row.keys():
            val = row[col]
            srs = srs & (df[col] == val)
        return df[srs]
    
def getPopulation(dataset, threshold=1.05, exclude_infeasible=True):
    pareto_frontier = getParetoFrontier(dataset, exclude_infeasible=exclude_infeasible)
    population = pd.DataFrame(columns=dataset.columns)
    for i,row in enumerate(dataset.iloc):
        is_near_optimal = checkParetoOptimal(row, pareto_frontier, threshold)
        # 3/8 add this, since in some cases infeasible points can also provide useful information
        if exclude_infeasible:
            is_feasible = row.is_feasible
        else:
            is_feasible = True
        is_error = row.is_error
        if(is_near_optimal and is_feasible and (not is_error)):
            population = population.append(row)
    return population

def importanceAnalysis(models, dataset, weights=[1, 0.4, 0.1, 0.1, 0.4]):
    columns = dataset.columns[:-7]
    importances_raw = np.zeros_like(regr_lat.feature_importances_)
    
    for i, regr in enumerate(models):
        importances_raw = importances_raw + regr.feature_importances_ * weights[i]
    
    importances = {}
    pos = 0
    for i,variable_name in enumerate(columns):
        if (re.match('^loop_.+_type$' ,variable_name)):
            importances[variable_name] = np.sum(importances_raw[pos:pos+3])/np.sum(weights)
            pos = pos + 3
        elif (re.match('^loop_.+_factor$' ,variable_name)):
            importances[variable_name] = np.sum(importances_raw[pos:pos+1])/np.sum(weights)
            pos = pos + 1
        elif (re.match('^array_.+_type$' ,variable_name)):
            importances[variable_name] = np.sum(importances_raw[pos:pos+3])/np.sum(weights)
            pos = pos + 3
        elif (re.match('^array_.+_factor$' ,variable_name)):
            importances[variable_name] = np.sum(importances_raw[pos:pos+1])/np.sum(weights)
            pos = pos + 1
    return importances

def importanceAdjustment(importances, gamma, prob_scale=1.5):
    def getMutationProb(normalized_importance, gamma):
        x = gamma
        mean = normalized_importance
        stdev = 0.3 # let's test fixed stdev first
        y = (x - mean)/stdev
        return scipy.stats.norm.pdf(y)*np.sqrt(2*np.pi)/2
    
    adjusted_importances = importances.copy()
    params = importances.keys()
    vals = list(importances.values())
    max_val = np.max(vals)
    min_val = np.min(vals)
    
    for param in params:
        importance = float(importances[param])
        
        # normalized importance
        normalized_importance = (importance - min_val)/(max_val - min_val)
        
        # get adjusted importance/ probability of mutation
        # adjusted_importance = getMutationProb(normalized_importance, gamma) * prob_scale
        adjusted_importance = 0.1
        adjusted_importances.update({param: adjusted_importance})
    
    return adjusted_importances

class BetaDistCounter():
    def __init__(self):
        self.n = 0
        self.a = 1  # the number of times this socket returned a charge        
        self.b = 1  # the number of times no charge was returned     
    
    def reset(self):
        self.__init__()
    
    def update(self,R):
        self.n += 1    
        self.a += R
        self.b += (1-R)
        
    def sample(self):
        return np.random.beta(self.a,self.b)
    
def selectMethod(method_records, view_boundary=30):
    # initialize the beta distributions of them
    random_beta = BetaDistCounter()
    evo_beta = BetaDistCounter()
    mut_beta = BetaDistCounter()
    records_in_view = method_records[-view_boundary:]
    for method, result in records_in_view:
        if(method == 'rand'):
            random_beta.update(result)
        elif(method == 'evo'):
            evo_beta.update(result)
        elif(method == 'mut'):
            mut_beta.update(result)
        else:
            raise(AssertionError('Unknown proposing method'))
    
    rand_beta_sample = random_beta.sample()
    evo_beta_sample = evo_beta.sample()
    mut_beta_sample = mut_beta.sample()
    print('Rand beta sample: '+str(rand_beta_sample))
    print('Evo beta sample: '+str(evo_beta_sample))
    print('Mut beta sample: '+str(mut_beta_sample))

    select = np.argmax([rand_beta_sample, evo_beta_sample, mut_beta_sample])
    print([rand_beta_sample, evo_beta_sample, mut_beta_sample])
    methods = ['rand', 'evo', 'mut']
    print(methods[select])
    return methods[select]

def predict_perf(parameters, models):
    regr_lat, regr_dsp, regr_ff, regr_lut, regr_bram, clss_timeout = models
    
    encoded_features = preprocessing(pd.Series(parameters).to_frame().T.to_numpy(), feature_columns)

    pred_lat = regr_lat.predict(encoded_features)
    pred_dsp = regr_dsp.predict(encoded_features)
    pred_ff = regr_ff.predict(encoded_features)
    pred_lut = regr_lut.predict(encoded_features)
    pred_bram = regr_bram.predict(encoded_features)
    proba_timeout = clss_timeout.predict_proba(encoded_features)[0,0]

    predicted_perf = {'latency':pred_lat,
                      'dsp_perc':pred_dsp,
                      'ff_perc':pred_ff, 
                      'lut_perc':pred_lut,
                      'bram_perc':pred_bram}
    return predicted_perf, proba_timeout

def randProposal(directives_path, no_partitioning):
    _, parameters = dir_gen.generate_directives(out_file_path=None, no_partitioning=no_partitioning)

    return parameters

def evoProposal(no_partitioning, importances, gamma, dataset, models, pareto_frontier, n_families=3, n_offsprings=3, threshold=1.0, exclude_infeasible=True):
    pareto_set = getPopulation(dataset, threshold=1.0, exclude_infeasible=exclude_infeasible).sort_values('latency')
    
    # select only 1 point for each unique latency randomly
    unique_latencies = pareto_set['latency'].unique()
    unique_pareto_points_idx = []
    for lat in unique_latencies:
        eq_lat_points = pareto_set[pareto_set['latency'] == lat]
        rand_idx = random.randint(0, len(eq_lat_points)-1)
        selected = eq_lat_points.index[rand_idx]
        unique_pareto_points_idx.append(selected)
    pareto_points = pareto_set.loc[unique_pareto_points_idx]
    
    latency = pareto_frontier['latency']
    min_lat = min(latency)
    max_lat = max(latency)

    # get the population
    population = getPopulation(dataset, threshold=1.2, exclude_infeasible=exclude_infeasible)

    list_params = []
    list_probs = []

    for i in range(n_families):
        parent_idx = np.random.randint(0, len(population),size=1)
        parent_rand = population.iloc[parent_idx]
        latency_ranking = latency.append(parent_rand['latency'])
        ranking = latency_ranking.rank(method='min')
        rank = int(ranking.iloc[-1])

        parent_lat = parent_rand['latency'].to_numpy()[0]
        
        if(parent_lat <= min_lat): # unlikely to happen, just in case
            print('case 1: selected the fastest')
            if(len(pareto_points) == 1): # edge case, only 1 in pareto set
                parent_pareto = pareto_points.iloc[0] # other parent is the next fastest one
            else:
                parent_pareto = pareto_points.iloc[1] # other parent is the next fastest one
        elif(parent_lat >= max_lat): # it's worse than the pareto points
            print('case 2: selected the slowest')
            parent_pareto = pareto_points.iloc[-2] # other parent is the second from the last
        else:
            print('case 3: selected a middle one')
            if(parent_lat in list(latency)): # in this case rank-1 will be the point it self
                print('selected does have pareto_latency')
                upper = pareto_points.iloc[rank] # neighboring point on the frontier with higher latency
                lower = pareto_points.iloc[rank-2] # neighboring point on the frontier with lower latency
            else:
                print('selected does not have pareto_latency')
                upper = pareto_points.iloc[rank-1] # neighboring point on the frontier with higher latency
                lower = pareto_points.iloc[rank-2] # neighboring point on the frontier with lower latency
            parent_pareto = random.choice([upper, lower])
        
        # parent_rand is a DF, parent pareto is a series
        parents = parent_rand.append(parent_pareto)

        for j in range(n_offsprings):
            _, offspring_parameters = crossover.generate_directives(out_file_path=None, 
                                                                    no_partitioning=no_partitioning, 
                                                                    context=parents)
            offspring_perf,_ = predict_perf(offspring_parameters, models)
            list_params.append(offspring_parameters)
            list_probs.append(getProbabilityOfEval(offspring_perf, pareto_frontier, threshold=threshold))

            _, mutant_parameters = mutator.generate_directives(out_file_path=None, 
                                                        no_partitioning=no_partitioning, 
                                                        context=(offspring_parameters, importances))
            mutant_perf,_ = predict_perf(mutant_parameters, models)
            list_params.append(mutant_parameters)
            list_probs.append(getProbabilityOfEval(mutant_perf, pareto_frontier, threshold=threshold))
            
    for i in range(1, len(list_params)+1):
        best = np.argsort(list_probs)[-i]
        proba_eval = list_probs[best]
        parameters = list_params[best]
        if (get_row(dataset, parameters).empty): 
            break
    return parameters, proba_eval

def mutProposal(no_partitioning, importances, gamma, dataset, models, pareto_frontier, n_mutants=3, threshold=1.2, exclude_infeasible=True):
    pareto_set = getPopulation(dataset, threshold=1.0, exclude_infeasible=exclude_infeasible)
    
    # select only 1 point for each unique latency randomly
    unique_latencies = pareto_set['latency'].unique()
    unique_pareto_points_idx = []
    for lat in unique_latencies:
        eq_lat_points = pareto_set[pareto_set['latency'] == lat]
        rand_idx = random.randint(0, len(eq_lat_points)-1)
        selected = eq_lat_points.index[rand_idx]
        unique_pareto_points_idx.append(selected)
    pareto_points = pareto_set.loc[unique_pareto_points_idx]

    rand_idx = random.randint(0, len(pareto_points)-1)
    _, mutant_parameters = mutator.generate_directives(out_file_path=None, 
                                            no_partitioning=no_partitioning, 
                                            context=(pareto_points.iloc[rand_idx], importances))
    mutant_perf,_ = predict_perf(mutant_parameters, models)
    prob_eval = getProbabilityOfEval(mutant_perf, pareto_frontier, threshold=threshold)
    return mutant_parameters, prob_eval

def update_models(models, dataset):
    regr_lat, regr_dsp, regr_ff, regr_lut, regr_bram, clss_timeout = models
    
    # extract features and labels from the feature set
    feature_columns = dataset.columns[:len(dataset.columns)-7]
    label_columns = dataset.columns[len(dataset.columns)-7:]
    features = dataset[feature_columns].to_numpy()
    labels = dataset[label_columns].to_numpy()
    features_encoded = preprocessing(features, feature_columns)

    # first determine the fesibility and timeout 
    timeout= labels[:,-1].astype('bool')

    lat = labels[:, 0]
    dsp_perc = labels[:, 1]
    ff_perc = labels[:, 2]
    lut_perc = labels[:, 3]
    bram_perc = labels[:, 4]

    not_timeout = np.logical_not(labels[:,-1].astype('bool'))
    
    # Notice that the regressions are trained only on points that do not timeout
    # Otherwise, these points will disturb the prediction 
    regr_lat.fit(features_encoded[not_timeout], lat[not_timeout])
    regr_dsp.fit(features_encoded[not_timeout], dsp_perc[not_timeout])
    regr_ff.fit(features_encoded[not_timeout], ff_perc[not_timeout])
    regr_lut.fit(features_encoded[not_timeout], lut_perc[not_timeout])
    regr_bram.fit(features_encoded[not_timeout], bram_perc[not_timeout])
    
    # timeout prediction will be trained on all points
    clss_timeout.fit(features_encoded, timeout)

def preprocessing(features, columns):
    # define the categories and encoders
    # the reason to use predefined categories is to avoid special cases, where some of the input features
    # only have one category present in the current dataset
    loop_directive_types = ['pipeline','unroll','none']
    array_directive_types = ['cyclic','block','complete','none']
    #enc_loop = OneHotEncoder(categories=[loop_directive_types], drop='first', sparse=False)
    #enc_array = OneHotEncoder(categories=[array_directive_types], drop='first', sparse=False)
    # for now, we do not drop the category
    enc_loop = OneHotEncoder(categories=[loop_directive_types], sparse=False)
    enc_array = OneHotEncoder(categories=[array_directive_types], sparse=False)
    
    #
    list_features_encoded = []
    #for i in range(features.shape[1]):
    for i,col in enumerate(columns):
        feature = features[:,i].reshape(1,-1).transpose()
        
        # detect data type of a feature
        if isinstance(feature[0][0], str):
            # identify the type of feature
            if (re.match('^loop_.+_type$' ,col)):
                encoder = enc_loop
            elif (re.match('^array_.+_type$' ,col)):
                encoder = enc_array
            else:
                raise AssertionError('unknown directive types')
            
            # encode the feature
            encoded = encoder.fit_transform(feature).astype('int')
            list_features_encoded.append(encoded)
        else:
            list_features_encoded.append(feature)

    encoded_features = np.concatenate(list_features_encoded, axis=1)
    return encoded_features

In [3]:
# # set constants
# parameter_file = './test_files/3dr_params.csv'
# directives_path = './rosetta_test/solution1/directives.tcl'
# template_path = './3dr_template.txt'
# top_function = 'rendering'
# part='xc7k70tfbv676-1'
# num_initial = 20
# no_partitioning = True

# set constants
parameter_file = './test_files/face_detect_reduced_params.csv'
directives_path = './test_files/face_detect_reduced_directives.tcl'
template_path = './face_detect_template.txt'
top_function = 'face_detect'
# part='xc7k70tfbv676-1'
# num_initial = 20
# no_partitioning = False

# set constants
# parameter_file = './test_files/spam_filter_params.csv'
# directives_path = './test_files/spam_filter_directives2.tcl'
# template_path = './spam_filter_template.txt'
# top_function = 'SgdLR'
#part='xc7k70tfbv676-1'
part='xc7z045-ffg900-2'
num_initial = 20
no_partitioning = False

# define the random directive generator
dir_gen = RandomDirectiveGenerator(parameter_file)
crossover = DirectiveCrossover(parameter_file)
mutator = DirectiveMutator(parameter_file)
writer = DirectiveWriter(parameter_file)

In [4]:
# create the empty dataframe
dataframe_columns = []
tunable_params = pd.read_csv(parameter_file)
list_columns = []
for i in range(len(tunable_params)):
    knob = tunable_params.iloc[i]
    name=knob['name']
    boundary = knob['range']
    dim = knob.at['dim']
    
    # to fix the NaN problem for dim, since dim doesn't exist for loops
    try:
        knob.at['dim']=int(dim)
    except ValueError:
        knob.at['dim']=int(0)
    
    # parse the file and find the column names, the naming MUST be consistent with generate_directives.py
    if(knob['type'] == 'loop'):
        list_columns.append('loop_'+name+'_type')
        list_columns.append('loop_'+name+'_factor')
    elif(knob['type'] == 'array'):
        if not no_partitioning:
            name=name+'_dim'+str(int(dim))
            list_columns.append('array_'+name+'_type')
            list_columns.append('array_'+name+'_factor')
        else:
            pass
    else:
        raise AssertionError('Unknow knob type')
        
# add the columns for HLS result
list_columns = list_columns+['latency', 'dsp_perc', 'ff_perc', 'lut_perc', 'bram_perc', 'is_feasible', 'is_error']

# initialize the datset to be empty (always)
dataset = pd.DataFrame(columns=list_columns)

# set the feature columns and label columns
feature_columns = dataset.columns[:len(dataset.columns)-7]
label_columns = dataset.columns[len(dataset.columns)-7:]

In [5]:
num_init=20
dataset = pd.read_csv('./face_detect_reduced_init.csv', index_col=0)[0:num_init]
#dataset = pd.read_csv('./Results/No_EA_Baseline/3dr_no_ea.csv', index_col=0)[0:95]
#dataset = pd.read_csv('./Results/AL+EA_first_test/3dr_first_run_ea.csv', index_col=0)
#dataset = pd.read_csv('./optical_flow.csv', index_col=0)[0:num_init]
#dataset = pd.read_csv('./opticalflow_regression.csv', index_col=0)
dataset

,loop_COPY_LOOP1_type,loop_COPY_LOOP1_factor,loop_COPY_LOOP2_type,loop_COPY_LOOP2_factor,loop_Filters_type,loop_Filters_factor,loop_Initialize2y_type,loop_Initialize2y_factor,array_coord_dim0_type,array_coord_dim0_factor,...,array_enable_list_dim0_factor,array__II_dim0_type,array__II_dim0_factor,latency,dsp_perc,ff_perc,lut_perc,bram_perc,is_feasible,is_error
0,none,0,pipeline,0,none,0,none,0,complete,0,...,0,cyclic,5,2295000.0,0.088889,0.131878,0.265613,0.200917,True,False
1,pipeline,0,pipeline,0,none,0,pipeline,0,complete,0,...,0,cyclic,125,inf,1.000000,1.000000,1.000000,1.000000,False,True
2,unroll,5,unroll,25,none,0,unroll,4,complete,0,...,0,none,0,2141000.0,0.085556,0.134501,0.301290,0.197248,True,False
3,pipeline,0,unroll,25,pipeline,0,pipeline,0,complete,0,...,0,none,0,inf,1.000000,1.000000,1.000000,1.000000,False,True
4,unroll,25,pipeline,0,pipeline,0,none,0,complete,0,...,0,none,0,inf,1.000000,1.000000,1.000000,1.000000,False,True
5,none,0,pipeline,0,pipeline,0,unroll,12,complete,0,...,0,none,0,inf,1.000000,1.000000,1.000000,1.000000,False,True
6,none,0,pipeline,0,none,0,none,0,complete,0,...,0,block,25,2295000.0,0.088889,0.133964,0.267932,0.196330,True,False
7,pipeline,0,pipeline,0,none,0,none,0,complete,0,...,0,cyclic,25,2006000.0,0.086667,0.132525,0.272525,0.196330,True,False
8,unroll,25,unroll,5,none,0,pipeline,0,complete,0,...,0,none,0,inf,1.000000,1.000000,1.000000,1.000000,False,True
9,unroll,25,none,0,pipeline,0,none,0,complete,0,...,0,cyclic,5,842000.0,0.112222,0.135764,0.289323,0.191743,True,False


In [6]:
# define the predictive models
regr_lat = RandomForestRegressor(random_state=42)
regr_dsp = RandomForestRegressor(random_state=42)
regr_ff = RandomForestRegressor(random_state=42)
regr_lut = RandomForestRegressor(random_state=42)
regr_bram = RandomForestRegressor(random_state=42)
clss_timeout = RandomForestClassifier(random_state=42)
models = [regr_lat, regr_dsp, regr_ff, regr_lut, regr_bram, clss_timeout]

# train the models
update_models(models, dataset)

In [16]:
total_steps = 150
ESCAPE_THRESHOLD = 1000 # after 1000 failed attempt to get a point worth evaluating, escape
N_OFFSPRING = 20
pareto_frontier = getParetoFrontier(dataset, exclude_infeasible=True)
step = 4

method_records = []

# define a global step for easier management
while True:
    print('Globa step: '+str(step))

    # select a point proposal method based on beta distribution
    method = selectMethod(method_records, view_boundary=30)

    if method == 'rand':
        print('Method: Random')
    elif method == 'evo':
        print('Method: Evolution')
    elif method == 'mut':
        print('Method: Mutation')
    else:
        raise(AssertionError('Unknown proposing method'))

    # gamma value go from 1 to 0
    gamma = min(1 - step / (total_steps), 1)
    print('Gamma: '+str(gamma))

    # importance analyis
    importances = importanceAnalysis([regr_lat, regr_dsp, regr_ff, regr_lut, regr_bram], dataset)

    # adjust the importances for better exploration
    adjusted_importances = importanceAdjustment(importances, gamma)

    # set a counter to detect if we are stuck and cannot find a good value
    escape_count = 0

    while (True):
        '''generate a new design point'''
        if method == 'rand':
            parameters = randProposal(directives_path, no_partitioning)

            # make predictions on the performance
            predicted_perf, _ = predict_perf(parameters, models)

            # calculate the probability of evaluating the point, based on it's quality (how close it is to the Pareto curve)
            # 1.5 seems to be a good value, however, 1.5 is also quite wide
            proba_eval = getProbabilityOfEval(predicted_perf, pareto_frontier, threshold=1.5)
        elif method == 'evo':
            parameters, proba_eval = evoProposal(no_partitioning, adjusted_importances, gamma, dataset, models, pareto_frontier)
        elif method == 'mut':
            parameters, proba_eval = mutProposal(no_partitioning, adjusted_importances, gamma, dataset, models, pareto_frontier)
        else:
            raise(AssertionError, 'None method selected')

        # make predictions on the performance
        predicted_perf, proba_timeout = predict_perf(parameters, models)

        # increment the escape count
        escape_count = escape_count + 1

        random_val = np.random.rand(1)[0]
        proba_total = proba_eval*proba_timeout
        print(predicted_perf)
        print('Design:')
        print(parameters)
        print('random value:'+str(random_val))
        print('probability from pareto: '+str(proba_eval))
        print('probability from timeout:'+str(proba_timeout))
        print('total probability of evaluation:'+str(proba_total))

        if ((random_val < proba_total) \
            and get_row(dataset, parameters).empty) \
            or escape_count > 1000:
                if(escape_count > 1000): 
                    print('escape!!')
                else: 
                    print('New design point found.')
                break

    print(parameters)
    # output the directives
    writer.generate_directives(out_file_path=directives_path, no_partitioning=no_partitioning,context=parameters)

    # evaluate the design point
    new_design_point = get_perf(template_path, 
                                directives_path, 
                                top_function, 
                                part, 
                                parameters, 
                                verbose=False,
                                timelimit=800)

    print(new_design_point)

    is_pareto_optimal = checkParetoOptimal(new_design_point, pareto_frontier, threshold=1.0)
    
    # check if it's already an existing cost+latency combination
    # if so it's not pareto optimal, this will eliminate some bias on mutation and evo
    # gives random more space 
    latency = new_design_point['latency']
    cost = 0.4*new_design_point['dsp_perc'] \
        +0.1*new_design_point['ff_perc'] \
        +0.1*new_design_point['lut_perc'] \
        +0.4*new_design_point['bram_perc']
    existing_point = pareto_frontier[(pareto_frontier['latency'] == latency) & (pareto_frontier['cost'] == cost)]
    if (len(existing_point) != 0):
        print('Existing pareto point!')
        is_pareto_optimal = False
    
    if(is_pareto_optimal):
        method_records.append((method, 1))
        print('//////////////////////////////////////////////////////')
        print('!!!!!!!!!!!!!!!Success: New Pareto Point!!!!!!!!!!!!!!')
        print('//////////////////////////////////////////////////////')
    else:
        method_records.append((method, 0))
        print('//////////////////////////////////////////////////////')
        print('$$$$$$$$$$$$$$$Fail: BAD BAD BAD Point$$$$$$$$$$$$$$$$')
        print('//////////////////////////////////////////////////////')

    # add the evaluated design point to current dataset
    dataset = dataset.append(new_design_point, ignore_index=True)

    # update pareto frontier after getting a new point
    pareto_frontier = getParetoFrontier(dataset, exclude_infeasible=True)
    print('New Pareto Frontier:')
    print(pareto_frontier)

    # save data instantly for easier test running
    dataset.to_csv('face_detect.csv')
    pareto_frontier.to_csv('face_detect_pareto.csv')
    

    # increment the global step
    step = step + 1

    # train the models
    update_models(models, dataset)
     
    with open('face_detect_method_records.pickle', 'wb') as f:
        pickle.dump(method_records, f)

    # break if needed
    if step >= 150: break

Globa step: 4
Rand beta sample: 0.7428007994884532
Evo beta sample: 0.015104392855682111
Mut beta sample: 0.7967685311707803
[0.7428007994884532, 0.015104392855682111, 0.7967685311707803]
mut
Method: Mutation
Gamma: 0.9733333333333334
{'latency': array([1992340.]), 'dsp_perc': array([0.08735556]), 'ff_perc': array([0.13452582]), 'lut_perc': array([0.27144826]), 'bram_perc': array([0.19679817])}
Design:
{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'pipeline', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'none', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'none', 'loop_Initialize2y_factor': 0, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_ty

{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'none', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'unroll', 'loop_Initialize2y_factor': 2, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'cyclic', 'array__II_dim0_factor': 25, 'latency': 304000.0, 'dsp_perc': 0.09222222222222222, 'ff_perc': 0.13486276303751144, 'lut_perc': 0.2874611161939616, 'bram_perc': 0.1871559633027523, 'is_feasible': True, 'is_error': False}
/////////////////////

Globa step: 8
Rand beta sample: 0.09818812435846468
Evo beta sample: 0.7054856830656537
Mut beta sample: 0.09853062284890547
[0.09818812435846468, 0.7054856830656537, 0.09853062284890547]
evo
Method: Evolution
Gamma: 0.9466666666666667
case 3: selected a middle one
selected does not have pareto_latency
case 3: selected a middle one
selected does not have pareto_latency
case 3: selected a middle one
selected does have pareto_latency
{'latency': array([511974.92]), 'dsp_perc': array([0.0954]), 'ff_perc': array([0.13358783]), 'lut_perc': array([0.28637031]), 'bram_perc': array([0.18919266])}
Design:
{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'none', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'none', 'loop_Initialize2y_factor': 0, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_ty

{'latency': array([445954.7]), 'dsp_perc': array([0.11783333]), 'ff_perc': array([0.16897708]), 'lut_perc': array([0.29437928]), 'bram_perc': array([0.19698165])}
Design:
{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'unroll', 'loop_COPY_LOOP2_factor': 25, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'unroll', 'loop_Initialize2y_factor': 2, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'cyclic', 'array__II_dim0_factor': 125}
random value:0.4788765329321596
proba

{'latency': array([368164.92]), 'dsp_perc': array([0.09686667]), 'ff_perc': array([0.13513166]), 'lut_perc': array([0.28719044]), 'bram_perc': array([0.18880734])}
Design:
{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'none', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'none', 'loop_Initialize2y_factor': 0, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'cyclic', 'array__II_dim0_factor': 25}
random value:0.9402966947422903
probabilit

...

{'loop_COPY_LOOP1_type': 'unroll', 'loop_COPY_LOOP1_factor': 25, 'loop_COPY_LOOP2_type': 'unroll', 'loop_COPY_LOOP2_factor': 25, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'unroll', 'loop_Initialize2y_factor': 2, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'cyclic', 'array__II_dim0_factor': 25, 'latency': 55822.0, 'dsp_perc': 0.09222222222222222, 'ff_perc': 0.13180695333943276, 'lut_perc': 0.2695699908508692, 'bram_perc': 0.1963302752293578, 'is_feasible': True, 'is_error': False}
Existing pareto poin

Globa step: 143
Rand beta sample: 0.05505694844241308
Evo beta sample: 0.23515456215907146
Mut beta sample: 0.06200245101101541
[0.05505694844241308, 0.23515456215907146, 0.06200245101101541]
evo
Method: Evolution
Gamma: 0.046666666666666634
case 3: selected a middle one
selected does not have pareto_latency
case 3: selected a middle one
selected does not have pareto_latency
case 3: selected a middle one
selected does have pareto_latency
{'latency': array([55821.94]), 'dsp_perc': array([0.09263333]), 'ff_perc': array([0.13176132]), 'lut_perc': array([0.27420974]), 'bram_perc': array([0.2000367])}
Design:
{'loop_COPY_LOOP1_type': 'unroll', 'loop_COPY_LOOP1_factor': 25, 'loop_COPY_LOOP2_type': 'unroll', 'loop_COPY_LOOP2_factor': 25, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'pipeline', 'loop_Initialize2y_factor': 0, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'ar

Globa step: 145
Rand beta sample: 0.21833451627061734
Evo beta sample: 0.21186071817182647
Mut beta sample: 0.1025376481009418
[0.21833451627061734, 0.21186071817182647, 0.1025376481009418]
rand
Method: Random
Gamma: 0.033333333333333326
{'latency': array([642850.]), 'dsp_perc': array([0.0933]), 'ff_perc': array([0.13376139]), 'lut_perc': array([0.27707013]), 'bram_perc': array([0.18818349])}
Design:
{'loop_COPY_LOOP1_type': 'none', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'pipeline', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'unroll', 'loop_Initialize2y_factor': 8, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_ty

{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'unroll', 'loop_COPY_LOOP2_factor': 25, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'pipeline', 'loop_Initialize2y_factor': 0, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'cyclic', 'array__II_dim0_factor': 25, 'latency': 302000.0, 'dsp_perc': 0.09222222222222222, 'ff_perc': 0.1349245196706313, 'lut_perc': 0.28935956084172004, 'bram_perc': 0.1871559633027523, 'is_feasible': True, 'is_error': False}
////////////////

{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'unroll', 'loop_COPY_LOOP2_factor': 25, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'unroll', 'loop_Initialize2y_factor': 6, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'cyclic', 'array__II_dim0_factor': 25, 'latency': 303000.0, 'dsp_perc': 0.09222222222222222, 'ff_perc': 0.13487419945105214, 'lut_perc': 0.2877630375114364, 'bram_perc': 0.1871559633027523, 'is_feasible': True, 'is_error': False}
//////////////////

In [5]:
num_initial = 20

# generate random values
# 1: initialization phase
for i in range(num_initial):
    print('Generate design points: {0}/{1}'.format(i, num_initial ))
    # generate a new design point that doesn't exist in the dataset0 
    while (True):
        # generate a new design point
        _, parameters = dir_gen.generate_directives(out_file_path=directives_path, no_partitioning=no_partitioning)
        
        # check if the design point is valid
        if (get_row(dataset, parameters).empty): 
            break
    
    print(parameters)
    
    # evaluate the design point
    new_design_point = get_perf(
                        template_path,  
                        directives_path, 
                        top_function, 
                        part, 
                        parameters, 
                        verbose=False,
                        timelimit=600)

    print(new_design_point)
    #new_design_point = pd.DataFrame.from_dict(parameters, orient='columns')

    # add the evaluated design point to current dataset
    dataset = dataset.append(new_design_point, ignore_index=True)
    #dataset = dataset.append(parameters, ignore_index=True)
    dataset.to_csv('face_detect_reduced_init.csv')
    
# make sure the is_error column is boolean type
dataset.is_error = dataset.is_error.astype('bool')

Generate design points: 0/20
{'loop_COPY_LOOP1_type': 'none', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'pipeline', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'none', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'none', 'loop_Initialize2y_factor': 0, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'cyclic', 'array__II_dim0_factor': 5}
{'loop_COPY_LOOP1_type': 'none', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'pipeline', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'none', 'loop_Filters_factor': 0, 'loop_In

HLS time elapsed: 600.0989672999999/600
HLS Synthesis Error.
Cannot find synthesis report
{'loop_COPY_LOOP1_type': 'none', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'pipeline', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'pipeline', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'unroll', 'loop_Initialize2y_factor': 12, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'none', 'array__II_dim0_factor': 0, 'latency': inf, 'dsp_perc': 1, 'ff_perc': 1, 'lut_perc': 1, 'bram_perc': 1, 'is_feasible': False, 'is_error': True}
Generate

{'loop_COPY_LOOP1_type': 'pipeline', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'pipeline', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'none', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'unroll', 'loop_Initialize2y_factor': 8, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'none', 'array__II_dim0_factor': 0, 'latency': 2146000.0, 'dsp_perc': 0.08666666666666667, 'ff_perc': 0.13182296431838975, 'lut_perc': 0.27182067703568163, 'bram_perc': 0.19724770642201836, 'is_feasible': True, 'is_error': False}
Generate design point

HLS time elapsed: 600.1284150000001/600
HLS Synthesis Error.
Cannot find synthesis report
{'loop_COPY_LOOP1_type': 'none', 'loop_COPY_LOOP1_factor': 0, 'loop_COPY_LOOP2_type': 'pipeline', 'loop_COPY_LOOP2_factor': 0, 'loop_Filters_type': 'none', 'loop_Filters_factor': 0, 'loop_Initialize2y_type': 'pipeline', 'loop_Initialize2y_factor': 0, 'array_coord_dim0_type': 'complete', 'array_coord_dim0_factor': 0, 'array_s0_dim0_type': 'complete', 'array_s0_dim0_factor': 0, 'array_s1_dim0_type': 'complete', 'array_s1_dim0_factor': 0, 'array_s2_dim0_type': 'complete', 'array_s2_dim0_factor': 0, 'array_values_dim0_type': 'complete', 'array_values_dim0_factor': 0, 'array_addr_list_dim0_type': 'complete', 'array_addr_list_dim0_factor': 0, 'array_enable_list_dim0_type': 'complete', 'array_enable_list_dim0_factor': 0, 'array__II_dim0_type': 'none', 'array__II_dim0_factor': 0, 'latency': inf, 'dsp_perc': 1, 'ff_perc': 1, 'lut_perc': 1, 'bram_perc': 1, 'is_feasible': False, 'is_error': True}
Generate de